# Integrating Fortran in Python

* External code can be linked into Python using extension modules 
* Extension modules in Python uses a C Python API
* Can be used as normal Python modules
* Implementing a Python extension module is hard...

```C
#include "Python.h"

// The calculation function

static PyObject* sum(PyObject *self, PyObject *args)
{
    double a;
    double b;

    // Parse input arguments

    if (!PyArg_ParseTuple(args, "dd", &a, &b)) 
        return NULL;

    // Do our computation

    double c = a + b;

    // Return the results

    return Py_BuildValue("d", c);
}

// Module function table.

static PyMethodDef
module_functions[] = {
    { "sum", sum, METH_VARARGS, "Calculate sum." },
    { NULL }
};

// Module initialisation

void
initcext(void)
{
    Py_InitModule3("cext", module_functions, "A minimal module.");
}
```

# There is an easier way - F2PY

* F2PY translates Fortran code and creates Python extension code
* Automatically passes Numpy Arrays as Fortran arrays
* Command line tool - Compiles and links modules automatically
* Hard to debug - Make sure existing code works before use

# Example 1 - simple.f90

```fortran
subroutine simple(a,b,c)

	real, intent(in) :: a, b
	real, intent(out) :: c

	c = a + b

end subroutine simple
```

 Convert Fortran code to Python extension module using f2py

In [ ]:
!f2py -m simple -c simple.f90

Import module in Python and print generated documentation

In [2]:
import simple
print(simple.__doc__)

This module 'simple' is auto-generated with f2py (version:2).
Functions:
  c = simple(a,b)
.


In [3]:
print(simple.simple.__doc__)

c = simple(a,b)

Wrapper for ``simple``.

Parameters
----------
a : input float
b : input float

Returns
-------
c : float



Execute code in extension module

In [4]:
a = 42
b = 42
c = simple.simple(a, b)
print(c)

84.0


# Example 2 - arr1.f90

```fortran
! A[r,s] * B[s,t] = C[r,t]
subroutine matrix_multiply(A,r,s,B,t,C)
	integer :: r, s, t
	real, intent(in) :: A(r,s)
	real, intent(in) :: B(s,t)
	real, intent(out) :: C(r,t)

	C = matmul(A,B)
end subroutine matrix_multiply
```

In [ ]:
!f2py -m arr1 -c arr1.f90

In [6]:
import arr1
print(arr1.__doc__)

This module 'arr1' is auto-generated with f2py (version:2).
Functions:
  c = matrix_multiply(a,b,r=shape(a,0),s=shape(a,1),t=shape(b,1))
.


In [7]:
print(arr1.matrix_multiply.__doc__)

c = matrix_multiply(a,b,[r,s,t])

Wrapper for ``matrix_multiply``.

Parameters
----------
a : input rank-2 array('f') with bounds (r,s)
b : input rank-2 array('f') with bounds (s,t)

Other Parameters
----------------
r : input int, optional
    Default: shape(a,0)
s : input int, optional
    Default: shape(a,1)
t : input int, optional
    Default: shape(b,1)

Returns
-------
c : rank-2 array('f') with bounds (r,t)



In [8]:
import numpy as np

A = np.ones((6,6), 'f', order='F') * 10.0
B = np.ones((6,6), 'f', order='F') * 20.0
C = np.zeros((6,6), 'f', order='F')

print("id of C before multiply =",id(C))

C = arr1.matrix_multiply(A, B)

print("id of C after multiply =",id(C))

print(C)

id of C before multiply = 47594111136912
id of C after multiply = 47594111137072
[[ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]]


# Example 3 - arr2.f90

```fortran
! A[r,s] * B[s,t] = C[r,t]
subroutine matrix_multiply2(A,r,s,B,t,C)
	integer :: r, s, t
	real, intent(in) :: A(r,s)
	real, intent(in) :: B(s,t)
	real, intent(inout) :: C(r,t)

	C = matmul(A,B)
end subroutine matrix_multiply2
```

In [ ]:
!f2py -m arr2 -c arr2.f90

In [10]:
import arr2
print(arr2.__doc__)

This module 'arr2' is auto-generated with f2py (version:2).
Functions:
  matrix_multiply2(a,b,c,r=shape(a,0),s=shape(a,1),t=shape(b,1))
.


In [11]:
print(arr2.matrix_multiply2.__doc__)

matrix_multiply2(a,b,c,[r,s,t])

Wrapper for ``matrix_multiply2``.

Parameters
----------
a : input rank-2 array('f') with bounds (r,s)
b : input rank-2 array('f') with bounds (s,t)
c : in/output rank-2 array('f') with bounds (r,t)

Other Parameters
----------------
r : input int, optional
    Default: shape(a,0)
s : input int, optional
    Default: shape(a,1)
t : input int, optional
    Default: shape(b,1)



In [12]:
A = np.ones((6,6), 'f', order='F') * 10.0
B = np.ones((6,6), 'f', order='F') * 20.0
C = np.zeros((6,6), 'f', order='F')

print("id of C before multiply =",id(C))

arr2.matrix_multiply2(A, B, C)

print("id of C after multiply =",id(C))

print(C)

id of C before multiply = 47594111136192
id of C after multiply = 47594111136192
[[ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]]


# Example 4 - matrix.f90

```fortran
module matrix

contains

! A[r,s] * B[s,t] = C[r,t]
subroutine matrix_multiply(A,r,s,B,t,C)
	integer :: r, s, t
	real, intent(in) :: A(r,s)
	real, intent(in) :: B(s,t)
	real, intent(out) :: C(r,t)

	C = matmul(A,B)
end subroutine matrix_multiply

! A[r,s] * B[s,t] = C[r,t]
subroutine matrix_multiply2(A,r,s,B,t,C)
        integer :: r, s, t
        real, intent(in) :: A(r,s)
        real, intent(in) :: B(s,t)
        real, intent(inout) :: C(r,t)

        C = matmul(A,B)
end subroutine matrix_multiply2

end module matrix
```

In [ ]:
!f2py -m matrix -c matrix.f90

In [14]:
import matrix
print(matrix.__doc__)

This module 'matrix' is auto-generated with f2py (version:2).
Functions:
Fortran 90/95 modules:
  matrix --- matrix_multiply(),matrix_multiply2().


In [15]:
print(matrix.matrix.__doc__)

c = matrix_multiply(a,b,[r,s,t])

Wrapper for ``matrix_multiply``.

Parameters
----------
a : input rank-2 array('f') with bounds (r,s)
b : input rank-2 array('f') with bounds (s,t)

Other Parameters
----------------
r : input int, optional
    Default: shape(a,0)
s : input int, optional
    Default: shape(a,1)
t : input int, optional
    Default: shape(b,1)

Returns
-------
c : rank-2 array('f') with bounds (r,t)
matrix_multiply2(a,b,c,[r,s,t])

Wrapper for ``matrix_multiply2``.

Parameters
----------
a : input rank-2 array('f') with bounds (r,s)
b : input rank-2 array('f') with bounds (s,t)
c : in/output rank-2 array('f') with bounds (r,t)

Other Parameters
----------------
r : input int, optional
    Default: shape(a,0)
s : input int, optional
    Default: shape(a,1)
t : input int, optional
    Default: shape(b,1)



Modules are better imported using the from * import statement

In [16]:
del(matrix)

In [17]:
from matrix import *

In [18]:
matrix.matrix_multiply2(A, B, C)
print(C)

[[ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]
 [ 1200.  1200.  1200.  1200.  1200.  1200.]]
